In [1]:
import os

os.environ["USE_PYGEOS"] = "0"

import geopandas as gpd
import pandas as pd
from odc.geo import BoundingBox
from coastlines.grids import PHILIPPINES_25, PHILIPPINES_CRS

In [2]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(115, 3, 130, 23, crs="EPSG:4326").to_crs(PHILIPPINES_CRS)
geom = PHILIPPINES_25.geojson(bbox=bounds)
tiles_projected = gpd.GeoDataFrame.from_features(geom, crs="EPSG:4326").to_crs(PHILIPPINES_CRS)

tiles_projected.explore()

In [3]:
# Get the Natural Earth country boundaries dataset
ne_url = "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip"
natural_earth_10m = gpd.read_file(ne_url)

In [5]:
# Prepare a dataset that contains continental coastlines with a 15 km buffer
country = natural_earth_10m[natural_earth_10m["ADMIN"] == "Philippines"].to_crs(PHILIPPINES_CRS)
# asia = natural_earth_10m[natural_earth_10m["CONTINENT"] == "Asia"]
# asia_coast = asia.dissolve().boundary.to_crs(PHILIPPINES_CRS)
# asia_coast_buffer = asia_coast.buffer(15_000)

# Pick out coasts that are within 50 km of Vietnam
coasts = country.buffer(50_000)
coasts.explore()

NameError: name 'PHILIPPINES_CRS' is not defined

In [6]:
# Use OpenStreetMap data for small islands excluded from NE coastlines
# Download the OSM global land polygons from https://osmdata.openstreetmap.de/data/land-polygons.html
# Manually edit the downloaded file "land_polygons.shp" to include only features of interest
# Save the coastline of interest as geojson file at "data/raw/osm_islands.geojson"
# Open the coastline geojson file using fiona
osm_islands_polygon = gpd.read_file("../data/raw/osm_islands.geojson")
osm_islands_polygon.set_crs("EPSG:4326")
osm_islands_polygon = osm_islands_polygon.to_crs(vietnam_epsg)

# Buffer polygons to 15 km
osm_islands_buffer = osm_islands_polygon.buffer(15_000)

In [7]:
# Include remaining very small islands with data from https://amti.csis.org/scs-features-map/
# Filter the downloaded file "All_SCS_Features.gpkg" using Rock = 'Y' and manually edit the file to include only features of interest
# Save the points as geojson file at "data/raw/amti_islands.geojson
# Read data
amti_islands_point = gpd.read_file("../data/raw/amti_islands.geojson")
amti_islands_point.set_crs("EPSG:4326")
amti_islands_point = amti_islands_point.to_crs(vietnam_epsg)

# Buffer points to 15 km
amti_islands_buffer = amti_islands_point.buffer(15_000)

In [8]:
# Define AOI as union of continental and island datasets
aoi = gpd.GeoDataFrame(pd.concat([osm_islands_buffer, amti_islands_buffer, vietnam_coasts], ignore_index=True), geometry=0).unary_union

In [9]:
# Clip tiles to AOI
vietnam_coastal_tiles_clipped = tiles_projected.clip(aoi)

In [16]:
# Dump a simple list of all tile_ids in a text tile
vietnam_coastal_tiles_clipped["idx"].to_csv("../data/raw/vietnam_tile_ids.txt", index=False, header=False)

In [ ]:
# These tiles can be removed safely
SKIP_TILES = [
    # Slivers and empty tiles
    "6,22",
    "6,23",
    "6,21",
    "7,24",
    "8,20",
    "9,23",
    "10,41",
    "10,42",
    "11,19",
    "11,21",
    "11,22",
    "12,22",
    "12,23",
    "12,38",
    "12,45",
    "13,18",
    "13,37",
    "14,18",
    "14,19",
    "14,21",
    "15,35",
    "15,37",
    "15,38",
    "16,34",
    "16,44",
    "16,48",
    "17,23",
    "17,25",
    "17,36",
    "17,47",
    "17,48",
    "18,21",
    "18,27",
    "18,28",
    "18,29",
    "18,30",
    "18,35",
    "19,21",
    "19,22",
    "19,24",
    "19,32",
    "19,33",
    "19,34",
    "20,27",
    "20,28",
    "22,35",
    "23,34",
    "24,18",
    "24,19",
    "24,37",
    "25,18",
    "25,20",
    "25,36",
    "25,38",
    "26,20",
    "26,36",
    "27,20",
    "28,18",
    "28,19",
    "28,20",
    "29,15",
    "29,18",
    "30,16",
    "30,18",
    "31,20",
    "31,21",
    "31,25",
    "32,18",
    "32,19",
    "32,20",
    "32,21",
    "32,25",
    "34,23",
    "34,24",
    # Island adjacent
    "33,23",
    "34,19",
    "35,19",
    # Platforms/reefs included in AMTI data which do not appear to contain land
    "23,37",
    "23,38",
    "24,35",
    "24,38",
    "25,35",
    "26,35",
    "27,18",
    "28,15",
    "28,17",
    "29,22",
    "30,17",
    "30,19",
    "31,19",
    "32,17",
    "32,22",
    "32,23",
    "33,17",
    "33,18",
    "33,20",
    "33,21",
    "34,20",
    "34,21",
    "35,20"
]

vietnam_coastal_tiles_clipped_filtered = vietnam_coastal_tiles_clipped[~vietnam_coastal_tiles_clipped["idx"].isin(SKIP_TILES)]
vietnam_coastal_tiles_clipped_filtered.explore()

In [ ]:
# Export the results as GeoJSON
vietnam_coastal_tiles_clipped_filtered.rename(columns={'idx': 'id'}, inplace=True)
vietnam_coastal_tiles_clipped_filtered.to_crs("EPSG:4326").to_file("../data/raw/vietnam_tiles.geojson", driver='GeoJSON')